In [1]:
from sklearn.metrics import confusion_matrix, classification_report

In [5]:
"""
INCOMPLETE, NOT READY TO BE RUN
"""
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Input, Flatten
from modeling_utils import NumpyDataGenerator


# Parameters

def train_model(metadata_location,
                model_architecture = None,
                data_dim=(128,128),
                batch_size=24,
                n_classes=10,
                training_folds = [10,2,3,4,5,6,7,8,9],
                validation_folds = [1],
                shuffle=True):
    """
    """
    params = {'dim': data_dim,
              'batch_size': batch_size,
              'n_classes': n_classes,
              'shuffle': shuffle}

    # Datasets
    metadata = pd.read_csv(metadata_location)
    if shuffle:
        metadata = metadata.sample(frac=1)
    id_to_file_mapping = dict(zip(metadata['fsID'],metadata['location']))
    train_data = metadata[metadata['fold'].isin(training_folds)]
    test_data = metadata[metadata['fold'].isin(validation_folds)]
    # Generators
    training_generator = NumpyDataGenerator(list(train_data['fsID']), list(train_data['classID']),id_to_file_mapping, **params)
    validation_generator = NumpyDataGenerator(list(test_data['fsID']), list(test_data['classID']), id_to_file_mapping, **params)

    # Design model
    model = Sequential()
    model.add(Dense(12, input_shape=(batch_size,128,128), activation='relu',name='main_input'))
    model.add(Dense(8, activation='relu',name='middle_layer'))
    model.add(Flatten())
    model.add(Dense(n_classes, activation='sigmoid',name='classifier'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    # Train model on dataset
    steps_per_epoch = np.ceil(len(metadata) / batch_size)
    validation_steps = np.ceil(len(list(test_data['classID']))/batch_size)
    model.fit_generator(generator=training_generator,
                        validation_data=validation_generator,
                        use_multiprocessing=True,
                        workers=6,
                        verbose=1,
                        steps_per_epoch=steps_per_epoch,
                        validation_steps=validation_steps,
                        epochs=1,
                        shuffle=True
                       )
    #Validate
    validation_generator_2 = NumpyDataGenerator(list(test_data['fsID']), list(test_data['classID']), id_to_file_mapping, **params)
    
    predictions = model.predict_generator(validation_generator_2,
                                      steps = validation_steps,
                                      verbose=True)
    y_pred = np.argmax(predictions, axis=1)
    print('Confusion Matrix')
    cm = confusion_matrix(list(test_data['classID']), y_pred[:len(list(test_data['classID']))])
    print(cm)
    print(np.unique(np.array(y_pred), return_counts=True))
    print(np.unique(np.array(test_data['classID']), return_counts=True))
    return y_pred, test_data['classID']

In [6]:
metadata_file = "/media/romulo/6237-3231/urban_sound_challenge/spectrograms_128_array/location_mapping.csv"

In [7]:
train_model(metadata_file)

TypeError: The added layer must be an instance of class Layer. Found: Tensor("input_1:0", shape=(24, 128, 128), dtype=float32)